In [26]:
import importlib
import io
import os
import re
import requests
import tempfile
import zipfile

import earthpy as et
import geopandas as gpd
import holoviews as hv
import hvplot as hv
import hvplot.pandas
import hvplot.xarray
import json
import laspy
import numpy as np
from pyproj import CRS
import rasterio
import rasterio.features
from rasterio.crs import CRS
import rioxarray as rxr
import rioxarray.merge as rxrm
from scipy.ndimage import binary_opening, binary_closing
import shapely
from shapely.geometry import shape
import whitebox

from shapely.ops import unary_union
from utils.process_lidar import convert_las_to_tif
from utils.process_lidar import clean_raster_rioxarray
from utils.process_lidar import export_lidar_canopy_tif
from utils.process_lidar import process_canopy_areas
from utils.process_lidar import process_lidar_to_canopy

# Prepare project directories
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME, 'treebeard')
lidar_dir = os.path.join(data_dir, 'lidar_tile_scheme_2020')
lidar_las_dir = os.path.join(data_dir, 'las files')
aoi_shapefiles = [
    'assets/areas/immediate_project/Zumwinkel_property.shp',
    'assets/areas/planning/Potential Project Area_1.shp',
    'assets/areas/planning/Potential Project Area.shp',
    'assets/areas/planning/ProjectOverview.shp'
]
os.makedirs(data_dir, exist_ok=True)
os.makedirs(lidar_dir, exist_ok=True)
os.makedirs(lidar_las_dir, exist_ok=True)

In [2]:
las_index_path = os.path.join(
    data_dir,
    lidar_dir,
    'lidar_index_cspn_q2.shp'
)

# Download LIDAR index tiles
# Specify the download URL for the LAS tile index if it exists
if not os.path.exists(las_index_path):
    las_index_url = ('https://gisdata.drcog.org:8443/geoserver/DRCOGPUB/'
             'ows?service=WFS&version=1.0.0&request=GetFeature&'
             'typeName=DRCOGPUB:lidar_index_cspn_q2&outputFormat=SHAPE-ZIP')
    
    # Download the ZIP file
    response = requests.get(las_index_url)
    response.raise_for_status()  # Check that the request was successful

    # Extract the ZIP file
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        zip_ref.extractall(lidar_dir)

las_index_gdf = (
    gpd.read_file(las_index_path).set_index('tile')
#    .loc[['N3W345']]
)

las_index_gdf = las_index_gdf.to_crs('EPSG:4269')
crs = las_index_gdf.crs

las_index_plot = las_index_gdf.hvplot(
    tiles = 'OSM',
    crs=las_index_gdf.crs,
    geo = True,
    line_color='black',
    line_width=2,
    fill_alpha=0
)
las_index_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [3]:
# Open project areas shapefile and plot
proj_zip_path = 'assets/project_areas_merged.zip'

with zipfile.ZipFile(proj_zip_path, 'r') as zip_ref:
    temp_dir = '/tmp/extracted_shapefile'  # You can specify any temporary directory
    zip_ref.extractall(temp_dir)
    
extracted_shapefile_path = temp_dir + '/'

proj_area_gdf = gpd.read_file(extracted_shapefile_path)

proj_area_gdf = proj_area_gdf.to_crs("EPSG:4326")

proj_area_plot = proj_area_gdf.hvplot(
    x='x',
    y='y',
    aspect='equal',
    tiles='OSM',
    geo=True,
    line_color='red',
    line_width=2,
    fill_alpha=0
)

proj_area_plot


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [x,y]

In [4]:
# Identify the tiles that intersect each project area
select_tiles_gdf = gpd.sjoin(las_index_gdf, proj_area_gdf, how='inner', predicate='intersects')

select_tiles_gdf.reset_index(drop=False)
select_tile_plot = select_tiles_gdf.hvplot(
    x='x',
    y='y',
    aspect='equal',
    geo=True,
    line_color='blue',
    line_width=2,
    fill_alpha=0,
    xaxis=None,
    yaxis=None
)

tile_proj_plot = select_tile_plot * proj_area_plot
#hv.save(tile_proj_plot, 'lidar_tile_plot.png')
tile_proj_plot

C:\Users\Pete\AppData\Local\Temp\ipykernel_32956\2425977346.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4269
Right CRS: EPSG:4326

  select_tiles_gdf = gpd.sjoin(las_index_gdf, proj_area_gdf, how='inner', predicate='intersects')


:Overlay
   .Polygons.I  :Polygons   [x,y]
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.II :Polygons   [x,y]

In [7]:
select_tiles_gdf = select_tiles_gdf.reset_index(drop=False)

# Generate list of all tiles per project area
tiles_by_area = select_tiles_gdf.groupby('Proj_ID')['tile'].apply(list).reset_index()
tiles_by_area

,Proj_ID,tile
0,Conifer Hill,"[N4W399, N4W397, N4W389, N4W396, N4W388, N4W29..."
1,Unnamed 1,[N4W264]
2,Unnamed 2,"[N4W381, N4W391]"
3,Zumwinkel,[N4W351]


In [33]:
las_path = r"C:\Users\Pete\earth-analytics\data\treebeard\las files\test\N4W351.las"

# Open your LAS file
las_file = laspy.read(las_path)
crs_wkt = las_file.header.parse_crs().to_wkt()
crs_wkt


'COMPOUNDCRS["NAD83(2011) / Colorado North (ftUS) + NAVD88 height - Geoid18 (ftUS)",PROJCRS["NAD83(2011) / Colorado North (ftUS)",BASEGEOGCRS["NAD83(2011)",DATUM["NAD83 (National Spatial Reference System 2011)",ELLIPSOID["GRS 1980",6378137,298.257222101,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["Degree",0.0174532925199433]],ID["EPSG",6318]],CONVERSION["unnamed",METHOD["Lambert Conic Conformal (2SP)",ID["EPSG",9802]],PARAMETER["Latitude of 1st standard parallel",40.7833333333333,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Latitude of 2nd standard parallel",39.7166666666667,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8824]],PARAMETER["Latitude of false origin",39.3333333333333,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8821]],PARAMETER["Longitude of false origin",-105.5,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8822]],PARAMETER["Easting at false origin",3000000,LENGTHUNIT["US survey foot",0.304800609601219],ID["EPSG",8826]],PARAMETER[

In [24]:
epsg_crs

'<bound method LasHeader.parse_crs of <LasHeader(1.4, <PointFormat(6, 0 bytes of extra dims)>)>>'

In [36]:
proj_area = proj_area_gdf[proj_area_gdf['Proj_ID'] == 'Zumwinkel']
las_folder_path = r"C:\Users\Pete\earth-analytics\data\treebeard\las files\test"

test = process_lidar_to_canopy("Zumwinkel", proj_area, las_folder_path, canopy_height=5)

ImportError: module process_lidar_to_canopy not in sys.modules

In [ ]:
# Process tiles for each project area
# Generate a dictionary of canopy TIFs for each project area
# This code can be modified to process LIDAR for multiple features in the input geodataframe

las_root_url = 'https://lidararchive.s3.amazonaws.com/2020_CSPN_Q2/'
canopy_dict = {}

# Specify the site index from the table above to process LIDAR for a single site.
# Otherwise, LIDAR will be processed for all sites
site_to_process = tiles_by_area[tiles_by_area['Proj_ID'] == 'Zumwinkel'].copy()
for index, row in site_to_process.iterrows():
    tiles = row['tile']
    proj_area_name = row['Proj_ID']
    sel_proj_area_gdf = proj_area_gdf[proj_area_gdf['Proj_ID'] == proj_area_name]
    # Download all tiles for project area, process, and clip/merge
    tile_agg = []
    print("Processing LIDAR for " + proj_area_name)
    for tile in tiles:
        file_name = tile + ".las"
        print("Processing LIDAR tile " + tile)
        tile_path = os.path.join(
            lidar_las_dir,
            file_name
        )
        download_url = las_root_url + tile + ".las"
        if not os.path.exists(tile_path):
            # Download the LAS file
            response = requests.get(download_url)

            # Check if the request was successful
            if response.status_code == 200:
                with open(tile_path, 'wb') as file:
                    file.write(response.content)
                print(f"File downloaded successfully and saved to {tile_path}")
            else:
                print(f"Failed to download file. Status code: {response.status_code}")

        # Output path for first returns DEM
        output_fr_tif = os.path.join(
            lidar_las_dir,
            tile +'_fr.tif'
        )
        if not os.path.exists(output_fr_tif):
            convert_las_to_tif(tile_path, output_fr_tif, "first")
        
        # Output path for ground DEM
        output_gr_tif = os.path.join(
            lidar_las_dir,
            tile +'_gr.tif'
        )
        if not os.path.exists(output_gr_tif):
            convert_las_to_tif(tile_path, output_gr_tif, "ground")

        # Process ground and first return data to canopy height
        fr_dem = rxr.open_rasterio(output_fr_tif)

        gr_dem = rxr.open_rasterio(output_gr_tif)

        canopy_dem = fr_dem - gr_dem

        # Set all values greater than 5 (canopy) to 1 and all values less than 5 (no canopy) to 0
        # Modify this value to adjust canopy height sensitivity
        canopy_dem.values[canopy_dem < 5] = 0
        canopy_dem.values[canopy_dem > 5] = 1
        canopy_dem.name = tile + "_Canopy"
        canopy_dem = canopy_dem.round()
        canopy_dem = canopy_dem.rio.write_crs("EPSG:6430", inplace=True)
        canopy_dem = canopy_dem.rio.reproject(CRS.from_epsg(4326))
        canopy_dem = canopy_dem.astype('float64')
        nodata_value = canopy_dem.rio.nodata
        if nodata_value is not None:
            canopy_dem = canopy_dem.where(~np.isclose(canopy_dem, nodata_value), 0)
        canopy_dem.rio.write_nodata(0, inplace=True)
        canopy_dem = canopy_dem.astype('int32')
        tile_agg.append(canopy_dem)
    print("Merging LIDAR tiles for " + proj_area_name)
    # Merge all tiles that intersect with the project area and clip to project area
    canopy_merged = rxrm.merge_arrays(tile_agg).rio.clip(sel_proj_area_gdf.geometry)
    canopy_dict[proj_area_name] = canopy_merged        

In [ ]:
for tile in tiles:
    file_name = tile + ".las"
    print("Processing LIDAR tile " + tile)
    tile_path = os.path.join(
        lidar_las_dir,
        file_name
    )
    download_url = las_root_url + tile + ".las"
    if not os.path.exists(tile_path):
        # Download the LAS file
        response = requests.get(download_url)

        # Check if the request was successful
        if response.status_code == 200:
            with open(tile_path, 'wb') as file:
                file.write(response.content)
            print(f"File downloaded successfully and saved to {tile_path}")
        else:
            print(f"Failed to download file. Status code: {response.status_code}")

    # Output path for first returns DEM
    output_fr_tif = os.path.join(
        lidar_las_dir,
        tile +'_fr.tif'
    )
    if not os.path.exists(output_fr_tif):
        convert_las_to_tif(tile_path, output_fr_tif, "first")
    
    # Output path for ground DEM
    output_gr_tif = os.path.join(
        lidar_las_dir,
        tile +'_gr.tif'
    )
    if not os.path.exists(output_gr_tif):
        convert_las_to_tif(tile_path, output_gr_tif, "ground")

    # Process ground and first return data to canopy height
    fr_dem = rxr.open_rasterio(output_fr_tif)

    gr_dem = rxr.open_rasterio(output_gr_tif)

    canopy_dem = fr_dem - gr_dem

    # Set all values greater than 5 (canopy) to 1 and all values less than 5 (no canopy) to 0
    # Modify this value to adjust canopy height sensitivity
    canopy_dem.values[canopy_dem < 5] = 0
    canopy_dem.values[canopy_dem > 5] = 1
    canopy_dem.name = tile + "_Canopy"
    canopy_dem = canopy_dem.round()
    canopy_dem = canopy_dem.rio.write_crs("EPSG:6430", inplace=True)
    canopy_dem = canopy_dem.rio.reproject(CRS.from_epsg(4326))
    canopy_dem = canopy_dem.astype('float64')
    nodata_value = canopy_dem.rio.nodata
    if nodata_value is not None:
        canopy_dem = canopy_dem.where(~np.isclose(canopy_dem, nodata_value), 0)
    canopy_dem.rio.write_nodata(0, inplace=True)
    canopy_dem = canopy_dem.astype('int32')
    tile_agg.append(canopy_dem)
print("Merging LIDAR tiles for " + proj_area_name)
# Merge all tiles that intersect with the project area and clip to project area
canopy_merged = rxrm.merge_arrays(tile_agg).rio.clip(sel_proj_area_gdf.geometry)
canopy_dict[proj_area_name] = canopy_merged        

In [24]:
# Create a vector binary mask for canopy

canopy_array = canopy_dict[proj_area_name]
# Load the TIF file using rioxarray
binary_mask = canopy_array.squeeze()  # Assuming the data is in the first band

# Create a mask where cell values are 1
mask = binary_mask == 1

# Get the affine transform from the raster data
transform = binary_mask.rio.transform()

# Extract shapes (polygons) from the binary mask
shapes = rasterio.features.shapes(mask.astype(np.int16).values, transform=transform)
polygons = [shape(geom) for geom, value in shapes if value == 1]

# Create a GeoDataFrame from the polygons
canopy_gdf = gpd.GeoDataFrame({'geometry': polygons})

In [25]:
# Prep data for vector processing. Make sure CRS is set to coordinate system with correct units.

crs = canopy_array.rio.crs

if canopy_gdf.crs is None:
    canopy_gdf = canopy_gdf.set_crs(canopy_array.rio.crs)

# Define EPSG:6430 CRS
epsg_6430 = '6430'

# Reproject to EPSG:6430
canopy_gdf = canopy_gdf.to_crs(epsg=epsg_6430)

In [26]:
site_boundary= proj_area_gdf[proj_area_gdf['Proj_ID'] == proj_area_name]
site_boundary = site_boundary.to_crs("EPSG:6430")

In [27]:
# Process canopy gaps
# Adjust "buffer_distance" to modify the buffer from canopy edge used to generate gap polygons
canopy_and_gaps_processed = process_canopy_areas(canopy_gdf, site_boundary, buffer_distance=5)

In [28]:
# Output shapefiles
canopy_gaps_calced_path = os.path.join(lidar_las_dir+'\\lidar_'+ proj_area_name + '_canopy_gaps_calced.shp')
canopy_and_gaps_processed[1].to_file(canopy_gaps_calced_path)
canopy_gdf_path = os.path.join(lidar_las_dir+'\\lidar_'+ proj_area_name + '_canopy.shp')
canopy_gdf.to_file(canopy_gdf_path)
buffered_canopy_path = os.path.join(lidar_las_dir+'\\lidar_'+ proj_area_name + '_buffered_canopy.shp')
canopy_and_gaps_processed[0].to_file(buffered_canopy_path)

C:\Users\Pete\AppData\Local\Temp\ipykernel_17948\1169739847.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  canopy_and_gaps_processed[1].to_file(canopy_gaps_calced_path)
c:\Users\Pete\miniconda3\envs\treebeard\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Gap_Size_Category' to 'Gap_Size_C'
  ogr_write(


In [29]:
# # Plot outputs of canopy and buffered gaps

# canopy_gaps_calced_to_plot = canopy_gaps_calced.to_crs("EPSG:4326")
# gaps_plot = canopy_gaps_calced_to_plot.hvplot(
#     x='x',
#     y='y',
#     aspect='equal',
#     geo=True,
#     line_color='blue',
#     line_width=2,
#     fill_alpha=.5,
#     width = 600,
#     height=600,
#     tiles = 'EsriImagery',
#     title = "Processed Canopy Gaps from LIDAR with 5' Buffer"
# )

# canopy_gdf_to_plot = canopy_gdf.to_crs("EPSG:4326")
# canopy_plot = canopy_gdf_to_plot.hvplot(
#     x='x',
#     y='y',
#     aspect='equal',
#     geo=True,
#     line_color='blue',
#     line_width=2,
#     fill_alpha=.5,
#     width = 600,
#     height=600,
#     tiles = 'EsriImagery',
#     title = "Processed Canopy from LIDAR"
# )

# gaps_plot + canopy_plot